In [12]:
import cv2

def detectar_camaras():
    index = 0
    arr = []
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        else:
            arr.append(index)
        cap.release()
        index += 1
    return arr

# Uso de la función
camaras_disponibles = detectar_camaras()
print("Cámaras disponibles:", camaras_disponibles)

Cámaras disponibles: [0]


In [ ]:
from ultralytics import YOLO
import cv2

# Instancia del modelo YOLO con los pesos especificados 
pesos_yolo = "yolov8n.pt" # Usamos la ruta relativa
model = YOLO(pesos_yolo)

# Verificar si la cámara está disponible
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se pudo acceder a la cámara.")
else:
    try:
        # Realiza la detección de objetos en el flujo de video de la cámara del computador 
        results = model(source=0, show=True, conf=0.3, save=True)
    except Exception as e:
        print("Ocurrió un error al realizar la detección de objetos:", e)
    finally:
        cap.release()

In [1]:
# Tracking
from ultralytics import YOLO 
# Configure the tracking parameters and run the tracker 
model = YOLO('yolov8n.pt') 
# Seleccionamos la clase 66 (keyboard)
results = model.track(source="0", conf=0.3, iou=0.5, show=True, classes=[66])  


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 62.3ms
0: 480x640 (no detections), 43.4ms
0: 480x640 (no detections), 39.3ms
0: 480x640 (no detections), 38.9ms
0: 480x640 (no detections), 40.4ms
0: 480x640 (no detections), 43.3ms
0: 480x640 (no detections), 44.1ms
0: 480x640 (no detections), 39.5ms
0: 480x640 (no detections), 38.2ms
0: 480x640 (no detections), 41.3ms
0: 480x640 (no detections), 40.2ms
0: 480x640

KeyboardInterrupt: 

WARNING  Video stream unresponsive, please check your IP camera connection.
WARNING  Video stream unresponsive, please check your IP camera connection.
